In [ ]:
import torch
import torchvision
torch.__version__




# 4.5 Multi-GPU parallel training

When we are training neural networks, because of the huge amount of calculation, a single GPU operation will make the calculation time very long, making us unable to get the results in time. For example, if we use but the GPU uses ImageNet data to train a classifier, it may be It takes a week or even a month. Therefore, a multi-GPU computing mechanism is introduced in Pytorch, so that the training speed can increase exponentially.

[DAWNBench](https://dawn.cs.stanford.edu/benchmark/) of Stanford University has recorded some of the records and implementation codes using multi-GPU computing so far, and those who are interested can take a look.

In this chapter, we will introduce three ways to use multi-GPU acceleration

## 4.5.1 torch.nn.DataParalle
Under normal circumstances, we will use a host with multiple graphics cards. This is the most budget-saving solution. Pytorch provides us with a very simple method to support but the host has multiple GPUs, then `torch.nn.DataParalle `We just need to pass in our own model as a parameter, and Pytorch will do the rest for us

In [ ]:
#Using a built-in model, here we take resnet50 as an example
model = torchvision.models.resnet50()

In [ ]:
#Model uses multiple GPUs
mdp = torch.nn.DataParallel(model)
mdp

With such a simple package, Pytorch has already done a lot of complicated work for us. We only need to increase our training batch_size (generally calculated as N times, N is the number of graphics cards), and other codes do not need to be changed.
Although the code does not need to be changed, the training convergence will be slow if the batch size is too large, so the learning rate must be increased. The university rate will also make the training of the model very unstable in the early stages, so a warm up of the learning rate is needed to stabilize the gradient drop, and then gradually increase the learning rate.

This kind of warm-up is only necessary for super large batches. Generally, our 4 cards or batch size less than 5000 (personal test) is basically unnecessary. For example, Fujitsu recently used 2048 GPUs and completed training in 74 seconds. The batch size used in the resnet50 experiment is 81920 [arivx](http://www.arxiv.org/abs/1903.12650).

The parallel processing mechanism of DataParallel is to first load the model on the main GPU (the default first GPU, GPU0 is the main GPU), then copy the model to each specified slave GPU, and then process the input data in batch dimensions Division, specifically, the number of data batches allocated to each GPU is the batch of total input data divided by the number of specified GPUs. Each GPU will perform forward calculation independently for its input data, and finally sum the loss of each GPU, and then use backpropagation to update the model parameters on a single GPU, and then copy the updated model parameters to the remaining specified GPUs In this way, an iterative calculation is completed.


DataParallel is actually a nn.Model, so the way we can save weights is no different from the general nn.Model, but if you want to use a single card or cpu for reasoning, you need to read the original model from it.

In [ ]:
#Get the original model
m=mdp.module
m

DataParallel will place the defined network model parameters on GPU 0 by default, so dataparallel can essentially be seen as copying the training parameters from the GPU to other GPUs while training, which will cause serious load imbalances in memory and GPU usage. Phenomenon, that is, the memory usage and usage rate of GPU 0 will greatly exceed the memory usage of other graphics cards, because here GPU0 is used as the master to summarize the gradients and update the model, and then send the calculation tasks to other GPUs, so his memory And the usage rate will be higher than others.

So we use the new torch.distributed to build a more synchronized distributed operation. Using torch.distributed can support not only a single machine but also multiple hosts and multiple GPUs for calculation.

## 4.5.2 torch.distributed
`torch.distributed` is a low-level API relative to `torch.nn.DataParalle`, so we need to modify our code to make it run independently on the machine (node). We want to be fully distributed and run processes independently on each GPU of each node, which requires N processes in total. N is the total number of our GPUs, here we use 4 to calculate.

First initialize the distributed backend, encapsulate the model, and prepare the data, which is used to train the process in an independent data subset. The modified code is as follows

In [ ]:
# The following script will definitely be unsuccessful when executed in jupyter notebook, please save it as a py file and test
import torch
import argparse
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader

# Here node_rank is the identifier of the local GPU
parser = argparse.ArgumentParser()
parser.add_argument("--node_rank", type=int)
args = parser.parse_args()

# Use Nvdea's nccl to initialize the node
torch.distributed.init_process_group(backend='nccl')

# Encapsulate the model on the GPU assigned to the current process
device = torch.device('cuda', arg.local_rank)
model = model.to(device)
distrib_model = torch.nn.parallel.DistributedDataParallel(model,
                                                          device_ids=[args.node_rank],
                                                          output_device=args.node_rank)

# Limit data loading to a subset of the data set (not including the current process)
sampler = DistributedSampler(dataset)

dataloader = DataLoader(dataset, sampler=sampler)
for inputs, labels in dataloader:
    predictions = distrib_model(inputs.to(device)) # forward propagation
    loss = loss_function(predictions, labels.to(device)) # calculate loss
    loss.backward() # Backpropagation
    optimizer.step() # optimization


At runtime, we can't simply use `python file name` to execute, we need to use the torch.distributed.launch prepared for us in PyTorch to run the script. It can automatically set the environment variables and call the script with the correct node_rank parameters.

Here we are going to prepare a machine as the master, all machines are required to be able to access it. Therefore, it needs to have an accessible IP address (in the example: 196.168.100.100) and an open port (in the example: 6666). We will use torch.distributed.launch to run the script on the first machine:
```bash
python -m torch.distributed.launch --nproc_per_node=2 --nnodes=2 --node_rank=0 --master_addr="192.168.100.100" --master_port=6666 file name (--arg1 --arg2 and other parameters)
```
Only need to change `--node_rank=0` on the second host

It is very likely that you will report an error when running, it is because we have not set the NCCL socket network interface
Let's take the network card named ens3 as an example, enter
```bash
export NCCL_SOCKET_IFNAME=ens3
```
The name ens3 can be checked and confirmed with the ifconfig command

Parameter Description:

--nproc_per_node: The total number of GPUs contained in the host

--nnodes: Total number of hosts

--node_rank: GPU identification in the host

Some other parameters can be viewed in [official document](https://github.com/pytorch/pytorch/blob/master/torch/distributed/launch.py)

torch.distributed not only supports nccl but also the other two backends gloo and mpi. The specific comparison is not detailed here, please check [official document](https://pytorch.org/docs/stable/distributed. html)

## 4.5.3 torch.utils.checkpoint
When we are training, we may encounter (I haven't encountered it yet) that a single sample of the training set is larger than the memory and cannot be loaded at all. So how do we train?

Pytorch provides us with gradient-checkpointing to save computing resources. Gradient checkpointing divides our continuously calculated meta forward and meta back propagation into fragments. However, due to the need to add additional calculations to reduce memory requirements, the efficiency of this method will be somewhat reduced, but it has obvious advantages in some examples, such as training RNN models on long sequences, which is difficult to reproduce. No introduction, the official document is [here](https://pytorch.org/docs/stable/checkpoint.html) Friends who encounter this situation can check the official solution.